In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import pandas_profiling
from pandas_profiling.base import describe
import numpy as np
from sklearn.preprocessing import Imputer
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import OneHotEncoder
import json


import json  

#利用panas读取文件函数
def createdf(filePath):
    df = pd.read_csv(filePath)
    return df

#生成数据预览静态文件，并返回分析对象profile
def createProfile(df,htmlFilePath):
    profile = pandas_profiling.ProfileReport(df)
    profile.to_file(outputfile=htmlFilePath)
    return profile

def createNVprofile(df):
    profileNV = describe(df,check_correlation=False)#生成不包含相关变量分析对象profile，profile为list格式
    return profileNV

#数据预览函数
def dataExplore(df):
    data_explore = df.head()
#     data_explore= df.sample(10)
    return data_explore#生成json，需要传到前端展示

#获取相关变量，并返回结果
def getCorrVar(profile):
    all_var = profile.description_set['variables']
    var_colums = all_var.columns.tolist()
    if(('correlation' in  var_colums) == True):
        rej_var = profile.get_rejected_variables()
        result = "{"+r'"'+"highCorr"+r'"'+":["
        for i, var_name in enumerate(rej_var):
            if (i != 0):
                result = result + ","
            #相关性
            result = result + "{"+r'"'+"corrValue"+r'"'+":"+r'"'+str(all_var.ix[var_name]['correlation'])+r'"'+","
            #变量
            result = result + ""+r'"'+"varName"+r'"'+":"+r'"'+var_name+r'"'+","
            #相关变量
            result = result + ""+r'"'+"corrVarName"+r'"'+":"+r'"'+all_var.ix[var_name]['correlation_var']+r'"'+"}"

        result= result + "]}"
    else:
        result ="{"+r'"'+"varName"+r'"'+":"+r'"'+"not exist correlation variable"+r'"'+"}"
    return  result
    



#选择要删除的相关变量
def HandleVarCorr(df,cols):
    if(len(cols)>0):
        cols = cols.split(',')
        for col in cols:
            df.drop([col],axis = 1,inplace=True)
    return df


#获取NULL变量
def getNullVar(profileNV):
    #需要用到Profile，所以不能在删除列后的基础上获取Null比率，这样得到的还是全部列的空值比例
#     all_var = profile.description_set['variables']#获取所有变量,返回值为DataFrame
    all_var = profileNV['variables']
    null_var = all_var.loc[all_var.p_missing >float(0)][['type','p_missing']]
    result = ""
    if(len(null_var)!=0):
        result = null_var.to_json()
    else:
        result ="{"+r'"'+"type"+r'"'+":"+r'"'+"not exist null variable"+r'"'+"}"
    return result

#  空值处理函数
def HandleVarNaN(df,col_input,constantvalue):
    for col in col_input:
        oper = col_input[col]
        if(oper == 'drop'):
            df.drop([col],axis = 1,inplace=True)
        elif(oper == 'mean'):
            df[col] = df[col].fillna(df[col].mean())
        elif(oper == 'median'):
            df[col] = df[col].fillna(df[col].median())
        elif(oper == 'max'):
            df[col] = df[col].fillna(df[col].max())
        elif(oper == 'min'):
            df[col] = df[col].fillna(df[col].min())
        elif(oper == 'zero'):
            df[col] = df[col].fillna(0)
        elif(oper == 'constant'):
            df[col] = df[col].fillna(constantvalue)
    return df    
            



#获取需要标准的变量
def getStandardVar(profileNV):
#     all_var = profile.description_set['variables']#获取所有变量,返回值为DataFrame
    all_var = profileNV['variables']
    scaler_var =all_var[['type','std','mini_histogram']]
    result = ""
    if(len(scaler_var)!=0):
        result = scaler_var.to_json()
    else:
        result = "{'type':'not exist need standard variable'}"
    return result



#数据处理函数，调用入口为StandardApply
# 需要在前端根据返回的需要标准化的变量，数值类（NUM）显示：包括Standarded、MinMax、MaxAbs、Robust，分类（CAT）：One-Hot Encoding
# ‘标准化’：‘Standarded’，‘最小-最大规范化’：‘MinMax’，‘绝对值最大标准化’：‘MaxAbs’，‘Robust标准化‘：‘Robust’，‘最小-最大规范化’：‘OneHotEncoding’

#StandardScaler
def Standarded(df,col,copy=True, with_mean=True, with_std=True):
    scaler = StandardScaler(copy,with_mean,with_std)
    newArray =scaler.fit_transform(df[col])
    df[col] = pd.DataFrame(newArray)
    df[col] = newArray.reshape(-1,1)
    return df

#MinMaxScaler
def MinMax(df,col,eature_range=(0, 1), copy=True):
    scaler =MinMaxScaler(eature_range,copy)
    newArray =scaler.fit_transform(df[col])
    df[col] = pd.DataFrame(newArray)
    return df

#MaxAbsScaler
def MaxAbs(df,col,copy=True):
    scaler =MaxAbsScaler(copy)
    newArray =scaler.fit_transform(df[col])
    df[col] = pd.DataFrame(newArray)
    return df

#RobustScaler
def Robust(df,col,with_centering=True, with_scaling=True, quantile_range=(25.0, 75.0), copy=True):
    scaler =RobustScaler(with_centering,with_scaling,quantile_range,copy)
    newArray =scaler.fit_transform(df[col])
    df[col] = pd.DataFrame(newArray)
    return df
def OneHotEncoding(df,col):
    ohe_var = pd.get_dummies(df[col])
    df.drop([col],axis = 1,inplace=True)
    df = df.join(ohe_var)
    return df

def StandardApply(df,cols_oper):
    for col in cols_oper:
        Standard = cols_oper[col]
        if(Standard=='Standarded'):
            df = Standarded(df,col)
        elif(Standard=='MinMax'):
            df=MinMax(df,col)
        elif(Standard=='MaxAbs'):
            df=MaxAbs(df,col)
        elif(Standard=='Robust'):
            df=Robust(df,col) 
        elif(Standard=='OneHotEncoding'):
            df=OneHotEncoding(df,col)
    return df

#获取工程的统计信息：
def projectDesc(df,profile):
    rows_count = df.count().max() # 行数
    cols_count = len(df.columns.tolist())# 列数
    
    all_var = profile.description_set['variables']
    num_count = all_var.loc[all_var['type'] == 'NUM'].count().max() # 数值型变量数
    cat_count = all_var.loc[all_var['type'] == 'CAT'].count().max() #类别型变量数
    null_scale  = float('%.2f' % (all_var['p_missing'].sum()/all_var['p_missing'].count()))#空值比例
    
    proDesc = {"rows_count":rows_count, "cols_count": cols_count, "num_count":num_count,"cat_count":cat_count,"null_scale":null_scale}
   
    return proDesc

In [ ]:
#step1:define file and htmlfilepath

filePath=
htmlFilePath=

df = createdf(filePath)
#利用pandas_profiling生成报告
profile = createProfile(df,htmlFilePath)
# step2:data explore （数据预览）
dataExplore(df)

In [ ]:
# step3:get correlation variables（获取相关变量）
getCorrVar(profile)

In [ ]:
#step4：delete choice correlation variables（删除已选择的相关变量）
deleteCols=
df = HandleVarCorr(df,deleteCols)
df.head()

In [ ]:
#step5：get null variables（获取空值变量）
profileNV = createNVprofile(df)
getNullVar(profileNV)

In [ ]:
# step6：fill null variables （空值处理）
# NUM 类型变量处理方式：'不处理': no,‘删除’:drop, '平均值':mean, '中位值':'median', '最大值':'max', '最小值':'min', '零值':'zero'
#其他类型变量处理方式：'不处理': no,‘删除’:drop,‘插入固定值’：‘constant’
#需要根据返回给前端的需要处理的空值变量类型判断，NUM类型显示：不处理',‘删除’, '平均值', '中位值', '最大值', '最小值', '零值',其他类型变量：‘不处理’,‘删除’,'插入固定值'
col_input=
constantvalue = 0
df = HandleVarNaN(df,col_input,constantvalue)
df.head()

In [ ]:
#step7：get Standard variables（获取需要标准化的变量）
profileNV = createNVprofile(df)
getStandardVar(profileNV)

In [ ]:
#step8：variables  standard（变量标准化）
col_input =
df = StandardApply(df,col_input)
#预览
dataExplore(df)

In [ ]:
#step9：save output file（保存输出文件）
outputFilePath="ConverData.csv"
df.to_csv(outputFilePath,index=False)

In [ ]:
#step10 :save project desc(工程描述性信息)
projectDesc(df,profile)